In [1]:
import os
import re
import xml.etree.ElementTree as ET

xml_files = [f for f in os.listdir('../../data/raw_data') if f.endswith('.xml')]

results = []

for i in range(min(50, len(xml_files))):  
    file_path = os.path.join('../../data/raw_data', xml_files[i]) 
    tree = ET.parse(file_path)  
    root = tree.getroot()  
    
    data = {}

    data["id"] = re.sub(r'\D', '', xml_files[i])

    # 출원 번호
    elem1 = root.find('.//biblioSummaryInfo')
    title = elem1.findtext('inventionTitle', '') if elem1 is not None else ''
    
    # 요약
    elem2 = root.find('.//abstractInfoArray/abstractInfo')
    abstract = elem2.findtext('astrtCont', '')  if elem2 is not None else ''
    
    data["embed"] = {'name':title, 'abstract':abstract}

    # IPC
    ipc = []

    for elem in root.findall('.//ipcInfoArray/ipcInfo/ipcNumber'):
        text = elem.text or ''
        ipc.append(text if text else '')

    # 우선권(국가)
    elem5 = root.find('.//priorityInfoArray/priorityInfo')
    country = elem5.findtext('priorityApplicationCountry', '') if elem5 is not None else ''

    # 공개/거절
    status = elem1.findtext('registerStatus', '') if elem1 is not None else ''

    # 링크
    elem6 = root.find('.//imagePathInfo')
    link = elem6.findtext('largePath', '') if elem6 is not None else ''

    data["metadata"] = {'ipc':ipc, 'priority':country, 'register':status, 'link':link}

    results.append(data)

In [2]:
for i in range(min(50, len(xml_files))):  

    # 1. 청구항 추출
    claims_raw = []
    for elem in root.findall('.//claimInfoArray/claimInfo/claim'):
        full = elem.text.strip()
        num = full.split('.')[0].strip()  # 청구항 번호
        value = full.split('.', 1)[1].strip()  # 청구항 내용
        claims_raw.append({"num": num, "text": value})

    # 2. 최종 청구항 정리
    final_claims = []

    for claim in claims_raw:
        num = claim["num"]
        text = claim["text"]
        
        # 종속항 패턴: 숫자와 항 사이 공백 가능
        m = re.search(r"제\s*(\d+)\s*항에\s*있어서", text)
        
        if m:
            parent_num = m.group(1)
            # 부모 독립항 찾기
            parent = next((c for c in final_claims if c["num"] == parent_num), None)
            clean_text = re.sub(r"제\s*\d+\s*항에\s*있어서[, ]*", "", text).strip()
            
            if parent:
                # 독립항 내용과 합치기
                combined_content = parent["content"] + " " + clean_text
                final_claims.append({
                    "num": num,
                    "content": combined_content,
                    "independent": False
                })
            else:
                # 부모 없으면 그냥 종속항 처리
                final_claims.append({
                    "num": num,
                    "content": clean_text,
                    "independent": False
                })
        else:
            # 독립항
            final_claims.append({
                "num": num,
                "content": text,
                "independent": True
            })

    # 3. 최종 embed에 넣기
    
    claims_for_embed = [{"content": c["content"], "independent": c["independent"]} for c in final_claims]
    results[i]["embed"]['claim'] = claims_for_embed

    results.append(data)


In [3]:
results

[{'id': '1020050108060',
  'embed': {'name': '정보 교류 기능을 가진 단말기, 이를 이용한 정보 교류시스템 및 방법',
   'abstract': ' 본 발명에 따른 단말기는 근거리 무선 통신을 통해 주변의 타 단말기 또는 서버와 정보를 교류할 수 있다. 또한 단말기 또는 서버는 자체에 데이터베이스를 구비하고 있어 타 단말기를 상대로 서비스 항목 정보를 교류 또는 GPS와 연계된 식별정보 등을 송수신할 수 있다. 본 발명의 특징적인 양상은 이 같은 단말기들이 주변의 타 단말기들의 식별정보, 위치정보와 같은 상황 정보를 직접 또는 서버로부터 수집하여 매 시각별로 저장함으로써 주변의 타 단말기 소지자들과 정보를 교류하거나 특정한 시각의 당시 주위 단말기 배치 상황을 조회할 수 있도록 할 수도 있다.단말기, 미팅, 리서치, 고유정보, 서비스 항목 정보, 전자 상거래, 식별정보, GPS, 지도, 상황도, GIS',
   'claim': [{'content': '조명 장치에 있어서,적어도 하나의 광원; 및상기 적어도 하나의 광원에 결합된 제어기로서, 시설 내의 상기 조명 장치의 위치에 기초해서 상기 적어도 하나의 광원의 출력을 제어하는, 상기 제어기를 포함하는, 조명 장치.',
     'independent': True},
    {'content': '청구항 1에 있어서, 상기 제어기는 적어도 하나의 센서에 결합되도록 구성되고, 상기 제어기는 상기 적어도 하나의 광원에 상기 적어도 하나의 센서에 의한 제1 환경 조건의 제1 검출에 기초하여 제1 광 출력을 출력하도록 지시하고, 상기 제어기는 상기 적어도 하나의 광원에 상기 적어도 하나의 센서에 의한 제2 환경 조건의 제2 검출에 기초하여 제2 광 출력을 출력하도록 지시하는, 조명 장치.',
     'independent': True},
    {'content': '청구항 2에 있어서, 상기 적어도 하나의 광원은 제1 광원 및 제2 광원을 포함하고, 상기 제1 광원은 상기 제1 

In [4]:
# json 확장자로 저장
import json

with open('preprocessed_data_independent.jsonl', 'w', encoding='utf-8') as f:
    for data in results:  
        f.write(json.dumps(data, ensure_ascii=False) + '\n')
